In [1]:
import numpy as np
import pandas as pd

from plots import plot_confusion_matrix, plot_roc_curve

In [2]:
import mlflow

# Para que funcione, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [3]:
!echo $AWS_ACCESS_KEY_ID
!echo $AWS_SECRET_ACCESS_KEY
!echo $MLFLOW_S3_ENDPOINT_URL

minio
minio123
http://localhost:9000


# MLFlow

Como hemos explorado en clase, MLFlow es una herramienta de código abierto diseñada para gestionar de manera eficiente el ciclo de vida completo de los modelos de Machine Learning. Este 
conjunto de funcionalidades incluye varios aspectos fundamentales:

- **Tracking**: Registra de forma sistemática los resultados y parámetros de los modelos durante su entrenamiento, permitiendo una comparación fácil y una comprensión más profunda de 
su rendimiento.
- **Projects**: Empaqueta el código de manera que sea completamente reproducible, facilitando la colaboración entre equipos y garantizando la portabilidad del código en diferentes entornos.
- **Models**: Proporciona herramientas para gestionar el versionado de los modelos, así como para desplegar modelos de ML como endpoints de servicio. Esta capacidad es especialmente 
valiosa, ya que MLFlow ofrece integraciones para el despliegue de modelos en la nube. Además, permite la exportación de modelos compatibles con Apache Spark.

## Instalación de MLFlow

Para este hands-on, utilizaremos Docker para configurar un entorno de MLFlow que nos permitirá acceder a todas las funcionalidades necesarias. Utilizaremos Docker Compose para orquestar 
no solo el servicio de MLFlow, sino también una base de datos PostgreSQL y buckets S3 utilizando MinIO. MLFlow utiliza tanto la base de datos como el bucket para almacenar 
información relevante sobre los modelos.

Para comenzar con este hands-on, ejecutamos el siguiente comando para levantar los servicios:

```Bash
docker-compose up 
```

Este comando garantizará que todos los servicios necesarios estén disponibles y listos para su uso en nuestro entorno de desarrollo de MLFlow.

## MLFlow Tracking

### Conceptos importantes sobre MLFlow Tracking

Guardar el tracking de nuestros modelos en nuestro servidor de MLflow es muy sencillo. Simplemente hay que hacer:

In [4]:
mlflow.set_tracking_uri('http://localhost:5001')

Con esto, ya tendremos hecha la conexión con nuestro servidor de MLFlow

Opcionalmente, podemos crear un experimento donde incluir los parámetros, si es que no lo tenemos ya. Cada proyecto diferente debería tener su propio experimento. Para ello, usaremos el 
método `create_experiment`:

In [5]:
experiment_name = "experiment_number_9"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name)

Existe los siguientes datos que podemos incluir en MLflow:

- **Parámetros del modelo**: Indica parámetros del modelo utilizado. Se registran usando el método `log_param`.
- **Métricas**: Se refiere a métricas de rendimiento, tales como el RMSE, accuracy, AUC, etc. Se registran usando el método `log_metric`.
- **Artefactos**: permite incluir archivos. El uso típico es incluir datos de entrenamiento, imágenes del entrenamiento, etc. Los artefactos se registran usando el método `log_artifact`.
- **Modelos**: permite incluir modelos. Los modelos se registran usando el método `log_model`. Además, MLFlow cuenta con tecnicas de auto-registro para las librerías Scikit-Learn, TensorFlow, 
Gluon, XGBoost, LightGBM, Statsmodels. Es decir, que se puede usar el método `autolog` y MLFlow automáticamente registrará los datos que vayamos generando. 

Para poder registrar, antes debemos indicar a MLFlow que *escuche*. Esto se puede hacer de dos formas:

1. Usar el método `star_run` junto con with para evitar cerrar el proceso.
```Python
with mlflow.start_run():
   mlflow.log_param('max_depth', max_depth)
```

2. Usar el método `start_run` y `end_run`
```Python
mlflow.start_run() 
mlflow.log_param('max_depth', max_depth) 
mlflow.end_run()
```

### Usando MLFlow con un ejemplo de Iris-Dataset

Vamos a emplear MLFlow para entrenar un modelo utilizando el conjunto de datos Iris. El modelo que utilizaremos será Random Forest de Scikit-Learn. Realizaremos una búsqueda 
de hiperparámetros mediante una búsqueda en cuadrícula y registraremos los mejores parámetros encontrados.

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [7]:
mlflow.set_tracking_uri('http://localhost:5001')

In [8]:
experiment_name = "experiment_iris"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name) 

experiment = mlflow.get_experiment_by_name(experiment_name)

In [9]:
# Cargo los datos
data = load_iris()

# Separamos entre evaluación y testeo
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=0.2, random_state=42)

In [10]:
# Armamos el modelo base
model = RandomForestClassifier()

In [11]:
# Definimos los hiperparámetros para la búsqueda
grid = {
    'max_depth':[6,8,10], 
    'min_samples_split':[2,3,4,5],
    'min_samples_leaf':[2,3,4,5],
    'max_features': [2,3]
    }

# Hacemos la búsqueda
iris_grid = GridSearchCV(model, grid, cv=5) 
iris_grid_results = iris_grid.fit(X_train, y_train)

print(f'Los mejores parámetros son: {iris_grid_results.best_params_}')

Los mejores parámetros son: {'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 2}


Con el mejor modelo, registramos toda la información:

In [12]:
with mlflow.start_run(experiment_id = experiment.experiment_id):
    # Se registran los mejores hiperparámetros
    mlflow.log_params(iris_grid_results.best_params_)
    
    # Se obtiene las predicciones del dataset de evaluación
    y_pred = iris_grid_results.predict(X_test)
    
    # Se calculan las métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    
    # Y las enviamos a MLFlow
    metrics ={
        'accuracy': accuracy,
        'precision': precision, 
        'recall': recall 
        }
    mlflow.log_metrics(metrics)
    
    # Como artefactos, obtenemos las gráficas de la curva ROC y la matriz de confusion
    matrix_plot = plot_confusion_matrix(y_test, y_pred, save_path=None)
    roc_plots = plot_roc_curve(y_test, y_pred, save_path=None)
    
    mlflow.log_figure(matrix_plot, artifact_file="matrix_plot.png")
    mlflow.log_figure(roc_plots[0], artifact_file="roc_curve_1_plot.png")
    mlflow.log_figure(roc_plots[1], artifact_file="roc_curve_2_plot.png")
    mlflow.log_figure(roc_plots[2], artifact_file="roc_curve_3_plot.png")
    
    # Registramos el modelo y los datos de entrenamiento
    mlflow.sklearn.log_model(iris_grid_results, 'iris_rf')

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


### Registrar un modelo

Una vez que hemos subido un modelo a MLFlow, ponerlo en producción es un proceso sencillo. Para ello, nos dirigimos a la pestaña *Artifacts* del modelo que deseamos implementar. Allí, hacemos click en el botón *Register Model*,, lo que registrará el modelo en el sistema.

Ahora, para obtener predicciones, tenemos dos opciones:

1. Leer el artefacto del servidor de MLFlow y lo usamos para hacer predicciones.
2. Publicar el artefacto como un endpoint.

#### Hacer predicciones de un modelo de MLFlow

Para obtener predicciones, vamos al apartado de *Artifacts* donde MLFlow indica cómo podemos obtener predicciones del modelo, ya sea utilizando Spark o Python. 
Copiamos ese código y lo ejecutamos, proporcionando los datos que deseamos predecir.

In [13]:
logged_model = 'runs:/402e88bd4bf14fca9e40f1cb9ad87212/iris_rf'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(pd.DataFrame(X_test))

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

El segundo formato permite desplegar un modelo mediante una REST API. Esto queda por fuera del hands-on, pero se puede consultar más [acá](https://mlflow.org/docs/latest/deployment/index.html).

Si queres ver más aplicaciones, podes ver la notebook del repositorio del proyecto final, el [ejemplo de aplicación](https://github.com/facundolucianna/amq2-service-ml/tree/example_implementation).